# My experimentation to incoperate object detection into our model

This notebook was one of the ways I tried to improve our model's accuracy. My thought was that if we could crop the person/people doing the poses, then we could reduce the noise in the images our CNN model had to classify.

There were a few challenges that came up as I tried to implement this into our pipeline to generate a model to classify the poses.  
You can read more about this at the bottom of the notebook.  

Note: All credit goes to this person github repo for providing for whatever I used in this notebook. I could not really remember where I got the rest of the code from, I'll update this notebook if I ever find out.  

If you are looking for the benchmark models notebook or what the team did for the final model, please look for those notebooks respectively.


---

# Setting up the notebook

This section will cover setting up the notebook with the repo

Mounting google drive into the notebook

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Cloning the github repo and setting up the directory required to run the set_up.py

In [0]:
!git clone https://github.com/matterport/Mask_RCNN.git

In [0]:
%cd {"Mask_RCNN"}
!python setup.py install

To check if the repo/module is properly installed.(I'm  not reallys sure what to call this)

In [0]:
!pip show mask-rcnn

# Loading the model and running first test

This is to load the model into the notebook, and test it on one picture.  
The next section will cover how to load in more images quickly.


This block of code will load in the pre-trained weights for the mask-rcnn model.  
This block will also test out the model by running it on one picture and generating the mask onto of the picture.  

Things to note:

* The reason I made this its own section is due to the long time taken to load the weights, so do not worry if this block takes a while to load.

* Make sure the directories lead to the proper files, look out for the comment I made in the code.

In [0]:
# example of inference with a pre-trained coco model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from matplotlib import pyplot
from matplotlib.patches import Rectangle
# example of inference with a pre-trained coco model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from mrcnn.visualize import display_instances
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
 
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
             'bus', 'train', 'truck', 'boat', 'traffic light',
             'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
             'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
             'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
             'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
             'kite', 'baseball bat', 'baseball glove', 'skateboard',
             'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
             'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
             'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
             'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
             'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
             'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
             'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
             'teddy bear', 'hair drier', 'toothbrush']
  
# draw an image with detected objects
def draw_image_with_boxes(filename, boxes_list):
     # load the image
     data = pyplot.imread(filename)
     # plot the image
     pyplot.imshow(data)
     # get the context for drawing boxes
     ax = pyplot.gca()
     # plot each box
     for box in boxes_list:
          # get coordinates
          y1, x1, y2, x2 = box
          # calculate width and height of the box
          width, height = x2 - x1, y2 - y1
          # create the shape
          rect = Rectangle((x1, y1), width, height, fill=False, color='red')
          # draw the box
          ax.add_patch(rect)
     # show the plot
     pyplot.show()
 
# define the test configuration
class TestConfig(Config):
     NAME = "test"
     GPU_COUNT = 1
     IMAGES_PER_GPU = 1
     NUM_CLASSES = 1 + 80
 
# define the model
# change the dirs under here based on how your files are structured. Make sure they are leading to their respective files
rcnn = MaskRCNN(mode='inference', model_dir='./', config=TestConfig())
# load coco model weights
rcnn.load_weights('/content/gdrive/My Drive/DSTA/mask_rcnn_coco.h5', by_name=True)
# load photograph
img = load_img('/content/gdrive/My Drive/DSTA/dataset/train_ver 1.0/data/ChairPose/013283_102.png')
img = img_to_array(img)
# make prediction
results = rcnn.detect([img], verbose=0)
# visualize the results
draw_image_with_boxes('/content/gdrive/My Drive/DSTA/dataset/train_ver 1.0/data/ChairPose/013283_102.png', results[0]['rois'])
# get dictionary for first prediction
r = results[0]
# show photo with bounding boxes, masks, class labels and scores
display_instances(img, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])

# Testing with more pictures

This section will cover how to load in more of your own pictures and generate the mask on top of the pictures.

Things to note:

* Same as before, make sure the directories lead to the correct files.


In [0]:
# make sure the directories are correct
img = load_img('/content/gdrive/My Drive/DSTA/dataset/train_ver 1.0/data/HandGun/073071_51.png')
img = img_to_array(img)
# make prediction
results = rcnn.detect([img], verbose=0)
# visualize the results
draw_image_with_boxes('/content/gdrive/My Drive/DSTA/dataset/train_ver 1.0/data/HandGun/073071_51.png', results[0]['rois'])
# get dictionary for first prediction
r = results[0]
# show photo with bounding boxes, masks, class labels and scores
display_instances(img, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])

# That's it

This was as far as I got to trying to implement the model into our team's pipeline.  

For me, this was pretty exciting to try and do as this was also my first time dealing with object detection as well as working extensively with images in python.  
I hope this notebook helps you to atleast start using object detection.

Some problems I encountered specific to our challenge of identifying poses:

* Some poses only showed the hands of the humans, such as handshake and handguns(to name a few).

* Some poses also required 2 people, such as chestbump

* Some data also had people in the background which were not part of the pose.